# (DON'T RUN) Automated generation of training data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import requests
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive/')
from tqdm import tqdm
import os
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Mounted at /content/gdrive/
cpu


In [ ]:
#sample_rate of the Playback dataset (from BABY Lab): 48000 (found out with torch.audio)
curr_dir =  "/content/gdrive/MyDrive/Evaluationofbabbling_ZhangEdelman/data/"
sample_rate = 48000
reference = pd.read_csv(curr_dir+str("Playback_infraphonology.csv"))
reference = reference[(reference["cat"]=="QRV")|(reference["cat"]=="FRV")|(reference["cat"]=="MSQR")|
                      (reference["cat"]=="MSFR")|(reference["cat"]=="CS")]

current_subject = -9999
for i, row in reference.iterrows():
  sub = row["sub"]
  if sub != current_subject:
    print(sub)
    current_subject = sub
    waveform, sample_rate = torchaudio.load(curr_dir +"Playback audio from BABY Lab"+
                                            "/PB S" + str(sub) + ".wav")

  start = row["onset"]
  end  = row["offset"]
  clip = waveform[0:1, int(start*sample_rate):int(end*sample_rate)]
  label = row["cat"]
  torchaudio.save(curr_dir + label+"/PB S" +str(sub)+"_"+str(i)+"_"+label+".wav", clip, sample_rate)

1
5
7
8
9
10
11
12
13
17
18
19
22
23
25
26
27
28
29
30
31
33
34
36
37
38
39
40
41
42
43
44
45
46
47
48
50
51
52
53
55
56
58
59
60
62
64
65
67
68
69
71
73
74
75
76
77



# Babble Classification with torchaudio


In [1]:
# CPU:
# !pip install pydub torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
# GPU:
# !pip install pydub torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import requests
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive/')
from tqdm import tqdm
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Mounted at /content/gdrive/
cpu


# Load data


In [2]:
def load_audio_files(path: str, label:str):
    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))
    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        #print("File_path", file_path)
        waveform, sample_rate = torchaudio.load(file_path)
        dataset.append([waveform, sample_rate, label])
    return dataset

In [3]:
curr_dir =  "/content/gdrive/MyDrive/Evaluationofbabbling_ZhangEdelman/data/"
LABELS = ["QRV", "FRV", "MSQR", "MSFR", "CS"]
all_data = []

for label in LABELS:
  dataset_label = load_audio_files(curr_dir + str(label), label)
  all_data.extend(dataset_label)

# Split into training and testing dataset

In [4]:
train_size = int(0.8 * len(all_data))
test_size = len(all_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(all_data, [train_size, test_size])

print("Training size:", len(train_dataset))
print("Testing size:",len(test_dataset))

Training size: 3064
Testing size: 766


In [5]:
waveform = train_dataset[0][0] # randomly picked an example
sample_rate = train_dataset[0][1]
print(f'Waveform: {waveform}')
print(f'Sample Rate: {sample_rate}')
print(f'Label: {train_dataset[0][2]}')


Waveform: tensor([[-0.0006, -0.0008, -0.0004,  ...,  0.0018,  0.0012, -0.0002]])
Sample Rate: 48000
Label: FRV


In [6]:
labels = sorted(list(set(datapoint[2] for datapoint in train_dataset)))
labels

['CS', 'FRV', 'MSFR', 'MSQR', 'QRV']

In [7]:
# Count number of each label
label_list = list(datapoint[2] for datapoint in train_dataset)
label_counts = dict()
for lab in label_list:
  if lab in label_counts:
    label_counts[lab] += 1
  else:
    label_counts[lab] = 0
print(label_counts)

{'FRV': 1042, 'QRV': 1352, 'MSQR': 84, 'CS': 337, 'MSFR': 244}


# Formatting the Data




In [8]:
## For the waveform, we downsample the audio for faster processing without losing too much of the classification power.
new_sample_rate = sample_rate #?????? not sure what is the best
transform = torchaudio.transforms.Resample(orig_freq=train_dataset[0][1], new_freq=new_sample_rate)
transformed = transform(waveform)
ipd.Audio(waveform.numpy(), rate=new_sample_rate)

In [9]:
# This is a function that changes word labels to numbers. NOTE: check if the numbers are treated weiredly

def label_to_index(word):
  if (word == "QRV"):
        return torch.tensor(0)
  #if (word == "MSFR"):
        #return torch.tensor(1)
  #if (word == "MSQR"):
        #return torch.tensor(1)
  #if (word == "MSCR"):
        #return torch.tensor(1)
  else:
        return torch.tensor(1)


word_start = label
index = label_to_index(word_start)

print(word_start, "-->", index)

CS --> tensor(1)


In [10]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros NOTE:
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
    tensors, targets = [], []
    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size= 64,
                                            shuffle=True, collate_fn= collate_fn, num_workers=0)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size= 64,
                                            shuffle=True, collate_fn=collate_fn, num_workers=0)



# Define the Network

Usually more advanced transforms are applied to the
audio data, however CNNs can be used to accurately process the raw data.
The specific architecture is modeled after the M5 network architecture
described in [this paper](https://arxiv.org/pdf/1610.00087.pdf)_.

In [11]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input = transformed.shape[0], n_output=35) #len(labels)) NOTE - changed
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=35, bias=True)
)
Numbe

We will use the same optimization technique used in the paper, an Adam
optimizer with weight decay set to 0.0001. At first, we will train with
a learning rate of 0.01, but we will use a ``scheduler`` to decrease it
to 0.001 during training after 20 epochs.




In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10

# (DON'T RUN) Load SpeechCommands dataset for pre-training

In [ ]:
from torchaudio.datasets import SPEECHCOMMANDS
import os

class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]


In [ ]:
# Create training and testing split of the data. We do not use validation in this tutorial.
sc_train_set = SubsetSC("training")
sc_test_set = SubsetSC("testing")

100%|██████████| 2.26G/2.26G [01:20<00:00, 30.3MB/s]


In [ ]:
sc_labels = sorted(list(set(datapoint[2] for datapoint in sc_train_set)))
sc_labels

['backward',
 'bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'follow',
 'forward',
 'four',
 'go',
 'happy',
 'house',
 'learn',
 'left',
 'marvin',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'visual',
 'wow',
 'yes',
 'zero']

In [ ]:
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
#transformed = transform(waveform)

In [ ]:
def sc_label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(sc_labels.index(word))


def index_to_sc_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return sc_labels[index]


word_start = "yes"
index = sc_label_to_index(word_start)
word_recovered = index_to_sc_label(index)

print(word_start, "-->", index, "-->", word_recovered)

yes --> tensor(33) --> yes


In [ ]:
def sc_collate_fn(batch):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
    tensors, targets = [], []
    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [sc_label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets


batch_size = 64

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

sc_train_loader = torch.utils.data.DataLoader(
    sc_train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=sc_collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
sc_test_loader = torch.utils.data.DataLoader(
    sc_test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=sc_collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

# (DON'T RUN) Pretrain on SpeechCommands dataset

In [ ]:
def sc_train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(sc_train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        #print("Data", data.shape, "output", output.shape, "target", target.shape)
        #print("First 20 outputs", output[0:20])
        #print("First 20 labels", target[0:20])

        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(sc_train_loader.dataset)} ({100. * batch_idx / len(sc_train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [ ]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def sc_test(model, epoch):
    model.eval()
    correct = 0
    for data, target in sc_test_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(sc_test_loader.dataset)} ({100. * correct / len(sc_test_loader.dataset):.0f}%)\n")

In [ ]:
log_interval = 20
n_epoch = 30

pbar_update = 1 / (len(sc_train_loader) + len(sc_test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        sc_train(model, epoch, log_interval)
        sc_test(model, epoch)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
# plt.plot(losses);
# plt.title("training loss");

  0%|          | 0.0006675567423230974/30 [00:00<10:29:58, 1259.98s/it]

Train Epoch: 1 [0/84843 (0%)]	Loss: 3.880428


  0%|          | 0.014018691588785041/30 [00:09<5:29:28, 659.26s/it]

Train Epoch: 1 [1280/84843 (2%)]	Loss: 3.509680


  0%|          | 0.02736982643524701/30 [00:15<4:08:38, 497.73s/it]

...

Train Epoch: 5 [74240/84843 (87%)]	Loss: 2.010059


 16%|█▌        | 4.78838451268323/30 [46:20<5:28:56, 782.82s/it]  

Train Epoch: 5 [75520/84843 (89%)]	Loss: 1.735420


 16%|█▌        | 4.7937249666218165/30 [46:23<4:03:54, 580.59s/it]


KeyboardInterrupt: 

In [ ]:
def predict(tensor):
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    tensor = transform(tensor)
    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_sc_label(tensor.squeeze())
    return tensor


waveform, sample_rate, utterance, *_ = sc_train_set[-1]
ipd.Audio(waveform.numpy(), rate=sample_rate)

print(f"Expected: {utterance}. Predicted: {predict(waveform)}.")

In [ ]:
torch.save(model.state_dict(), "sc_pretrained_model.ckpt")

# Make a copy of the network, modify the last layer, make a new optimizer

In [ ]:
model = M5(n_input = transformed.shape[0], n_output=35)
#model.load_state_dict(torch.load("sc_pretrained_model.ckpt", map_location=device))


In [ ]:
# Make a copy of the model for finetuning
import copy
finetuned_model = copy.deepcopy(model)
num_ftrs = finetuned_model.fc1.in_features
# Here the size of each output sample is set to 2 (number of classes in new dataset)
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
finetuned_model.fc1 = nn.Linear(num_ftrs, 2)

In [ ]:
# Create a new optimizer on the copied model's parameters
finetuned_optimizer = optim.Adam(finetuned_model.parameters(), lr=0.01, weight_decay=0.0001)
finetuned_scheduler = optim.lr_scheduler.StepLR(finetuned_optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10


# Training and Testing the Network

Training function will perform the backward pass and optimization steps. For
training, the loss we will use is the negative log-likelihood. The
network will then be tested after each epoch to see how the accuracy
varies during the training.




In [ ]:
def train(model, epoch, log_interval):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(trainloader):

        data = data.to(device)
        target = target.to(device)
        #print("Train data shape", data.shape)
        #print("Target shape", target.shape)
        # apply transform and model on whole batch directly on device
        #data = transform(data) * 100
        output = model(data)
        #print('\n======== data ======== \n')
        #print(data.shape)
        #print(data[0, 0, 20000:20050])
        # print('==================== \n')
        # print("Data", data.shape)
        # print("Output", output.shape, output)
        # print("Target", target.shape, target)
        #print(output)
        #print("Exp", torch.exp(output)) ## To make the model less biased towards extreme values, use "label smoothing"

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        finetuned_optimizer.zero_grad()
        loss.backward()
        finetuned_optimizer.step()

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)
        #print(output[0:5], target[0:5])

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(trainloader.dataset)} ({100. * batch_idx / len(trainloader):.0f}%)]\tLoss: {loss.item():.6f} \tAccuracy: {correct}/{len(trainloader.dataset)} ({100. * correct / len(trainloader.dataset):.0f}%)")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [ ]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in testloader:

        data = data.to(device)
        target = target.to(device)
        #print("Test data shape", data.shape)
        #print("Target shape", target.shape)

        # apply transform and model on whole batch directly on device
        #data = transform(data) * 100
        output = model(data)
        #print("Predictions", torch.exp(output))
        #print("Label", target)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(testloader.dataset)} ({100. * correct / len(testloader.dataset):.0f}%)\n")
    return 100. * correct / len(testloader.dataset)

In [ ]:
### The output is [probability of being 0, probability of being 1]


log_interval = 20
n_epoch = 50


pbar_update = 1 / (len(trainloader) + len(testloader))
losses = []
best_test_acc = -9999
best_epoch = None

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(finetuned_model, epoch, log_interval)
        test_acc = test(finetuned_model, epoch)
        finetuned_scheduler.step()
        if test_acc > best_test_acc:
          best_test_acc = test_acc
          best_epoch = epoch
          torch.save(finetuned_model.state_dict(), "trained_model_best.ckpt")


# Plot the training loss versus the number of iteration.
plt.plot(losses);
plt.title("training loss")

  0%|          | 0.016666666666666666/50 [00:03<2:58:36, 214.40s/it]

Train Epoch: 1 [0/3064 (0%)]	Loss: 0.724904 	Accuracy: 29/3064 (1%)


  1%|          | 0.35/50 [00:33<1:08:05, 82.30s/it]              

Train Epoch: 1 [1280/3064 (42%)]	Loss: 0.621495 	Accuracy: 782/3064 (26%)


  1%|▏         | 0.6833333333333338/50 [01:03<1:29:42, 109.14s/it]

...


Test Epoch: 28	Accuracy: 537/766 (70%)



 56%|█████▌    | 28.016666666666357/50 [35:53<16:31, 45.12s/it]

Train Epoch: 29 [0/3064 (0%)]	Loss: 0.451616 	Accuracy: 46/3064 (2%)


 56%|█████▋    | 28.216666666666345/50 [36:10<27:55, 76.94s/it]


KeyboardInterrupt: 

In [ ]:
def predict(tensor):
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    tensor = transform(tensor)
    tensor = finetuned_model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    return tensor


waveform, sample_rate, utterance, *_ = train_dataset[-1]
ipd.Audio(waveform.numpy(), rate=sample_rate)

print(f"Expected: {utterance}. Predicted: {predict(waveform)}.")

In [ ]:
for i, (waveform, sample_rate, utterance, *_) in enumerate(test_dataset):
    output = predict(waveform)
    if output != utterance:
        ipd.Audio(waveform.numpy(), rate=sample_rate)
        print(f"Expected: {utterance}. Predicted: {output}.")
        break
    else:
        print("All examples in this dataset were correctly classified!")
        print("In this case, let's just look at the last data point")
        ipd.Audio(waveform.numpy(), rate=sample_rate)
        print(f"Expected: {utterance}. Predicted: {output}.")

In [ ]:
# Save model to a file
torch.save(finetuned_model.state_dict(), "trained_model081324.ckpt")

# Visualization

In [ ]:
def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)